In [4]:
# Pkg.add("GraphPlot")
using JuMP
using LightGraphs
using GraphPlot

function plotGraph(edge)
    global N, origin, destination
#     n = maximum(edge)
#     println(n)
    g = SimpleDiGraph(n)
    label1 = collect(1:n)
    for i = 1:length(edge[:,1])
        add_edge!(g, edge[i,1], edge[i,2]);
    end
    gplot(g, layout=circular_layout, nodelabel=label1)
#     kruskal_mst(g, ones(length(edge[:,1])))
#     println(is_strongly_connected(g))
end
function gx_bound(c, c_g, edge, source_sink)
    global  N, origin, destination
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f, "Current interdiction x = ", x_now)
    if source_sink == true
        start_node = edge[:,1]
        end_node = edge[:,2]
        origin = 1
        destination = N
    else 
        start_node = edge[:,2]
        end_node = edge[:,1]
        origin = N
        destination = 1
    end
    no_node = max(maximum(start_node), maximum(end_node) )
    no_link = length(start_node)


    function getShortestX(state, start_node, end_node, origin, destination)
        _x = zeros(Int, length(start_node))
        _path = enumerate_paths(state, destination)

        for i=1:length(_path)-1
            _start = _path[i]
            _end = _path[i+1]

            for j=1:length(start_node)
                if start_node[j]==_start && end_node[j]==_end
                _x[j] = 1
                break
                end
            end

        end
        _x
    end


    graph = Graph(no_node)
    distmx = Inf*ones(no_node, no_node)

    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end

    # Run Dijkstra's Algorithm from the origin node to all nodes
    state = dijkstra_shortest_paths(graph, origin, distmx)
    label = state.dists
    pred = state.parents
    b_arc = ""
    
    for i = 1: length(state.parents)
        if state.parents[i] != 0 
            b_arc = string(b_arc, "(", state.parents[i], ",", i, ")")
        end
    end
    
    # Retrieving the shortest path
    path = enumerate_paths(state, destination)
    
    #parents = LightGraphs.DijkstraState(state, destination)

    # Retrieving the 'x' variable in a 0-1 vector
    y = getShortestX(state, start_node, end_node, origin, destination)
    #println(f,"y vector:", y)
    
    gx = sum(c_g[i]*y[i] for i = 1:no_link)    
    SP = sum(c[i]*y[i] for i = 1:length(c))
    T = Int64[]
    for i = 1:Len
        if pred[edge[i,2]] == edge[i,1]
            push!(T, 1)
        else
            push!(T, 0)
        end
    end
    
    origin = 1
    destination = N
#     println("Minimum Spanning Tree: ", T)
    #println("Shortest path y = ", y)
    #println("Indices of shortest path (edge) = ", y_index)
    #println("Nodes visited =", path)
#     println("Minimum Spanning Tree =", pred)
#     println("Node label = " ,label)

    return y, gx, SP, label
end

# function print2File()

# edge = [1 4; 2 6; 2 8; 2 10; 2 11; 2 15; 3 11; 4 13; 5 3; 5 13; 6 8; 7 2; 7 11; 7 15; 8 4; 8 7; 8 12; 8 15; 9 2; 9 6; 9 8; 9 13; 10 11; 11 10; 12 2; 12 5; 12 6; 12 11; 13 2; 13 7; 13 14; 14 5; 14 9; 14 13]
# edge = [1 2; 1 3; 3 2]
# global Len = length(edge[:,1])
# c = ones(Len)
# c_g = ones(Len)
# global origin = 1
# global destination = 3

# plotGraph(edge)
# y, gx, SP = gx_bound(c, c_g, edge)

gx_bound (generic function with 1 method)

In [7]:

# Pkg.add("GraphPlot")
# num = "14"
# edge = Int64[]
global edge = Array{Int64}(undef, 0, 2)
global N = 50
global origin = 1
global destination = N
global Len = 0
global b_x = 10
global b_y = 10
# global uncNum = 0.1 # 0.2
for num = 1:5 #0
    global edge, N, origin, destination, Len, uncNum
    print("",num,"...")
    stop_criterion = false
    density = 0.25
    while stop_criterion == false
        edge = Array{Int64}(undef, 0, 2)
        head = zeros(N)
        tail = zeros(N)
            for u = 1 : N-1
                for v = 2 : N
                    x = rand()
                    if x <= density && u != v 
                        edge = vcat(edge,[u v])
                    end
                end
            end
#         println("",edge)
        Len = length(edge[:,1])
        c_temp = ones(Len)
        
        #Verify there is a path from source to every node and from every node to sink
        try
            print("x")
            y, gx, SP, label1 = gx_bound(c_temp, c_temp, edge, true)
    #         println("label1 = ", label1)
            y, gx, SP, label2 = gx_bound(c_temp, c_temp, edge, false)
            if isfinite(maximum(label1))==true && isfinite(maximum(label2)) == true
                stop_criterion = true
            end
        catch
            stop_criterion = false
        end
    end
#     println(edge)
#     println(length(edge[:,1]))
#     plotGraph(edge)
    A = Len
    #Generate q-values as detection probability in (0,1) , 3 decimals
    q = Float64[]
    for i = 1 : A
        x = rand(1:999)/1000
        if x<0
            x=0
        end
        push!(q, x)
    end
    println("y")
#     println(cL)
#     println(cU)
    # println(myindex)
    origin = 1
    destination = N
    
#     println("Here:", Len) 
    y, gx, SP, label = gx_bound(q, q, edge, true)
    # println(gx)
#     println("HERE")
    d_x = Float64[]
    d_y = Float64[]
    for i = 1 : A
        x = rand(1:10)
        y = rand(1:10)
        push!(d_x, x)
        push!(d_y, y)
    end
    # println(d)
    d_name = int(density*100)
    outfile = "./N"*string(N)*"_d"*string(d_name)*"_"*string(num)*".jl"
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(f,"arcs = ", edge) #length(df[:CELL]))
    println(f,"d_x = ", d_x)
    println(f,"b_x = ", b_x)
    println(f,"d_y = ", d_y)
    println(f,"b_y = ", b_y)
    
    println(f,"q = ", q)
    println(f,"origin = 1")
    println(f,"destination = ", N)
    close(f)
end

1...xy
2...xy
3...xy
4...xy
5...xy


In [51]:
import Statistics
set_ins = rand(1:200,20)
temp_ = rand(20)
println(Statistics.mean(temp_[1:5]))
# average()
 for num in set_ins
#     include("./N50_Ins_"*string(num)*".jl")
#     A = length(edge)
#     println(num, ". min cL", minimum(cL_orig), "; max M ", maximum(M_orig[findall(M_orig.<20)]), "; max d ", maximum(d))
#     println("density = ", A/destination/(destination-1))
# end

0.5406004455810869


LoadError: [91msyntax: incomplete: premature end of input[39m

In [34]:
edge = [1 2; 2 3; 1 3]
c_g = [1, 1.5, 10]
function dijkstra(c_g, edge)
    no_node = maximum(edge)
    no_link = length(edge[:,1])
    start_node = edge[:,1]
    end_node = edge[:,2]
    
    graph = SimpleDiGraph(no_node)
    distmx = Inf*ones(no_node, no_node)

# Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end
    A = dijkstra_shortest_paths(graph, origin, distmx)
    return A
end
A = dijkstra(c_g, edge)

LightGraphs.DijkstraState{Float64,Int64}([0, 1, 2], [0.0, 1.0, 2.5], Array{Int64,1}[[], [], []], [1.0, 1.0, 1.0], Int64[])

In [59]:
#Generate New Files Unc Range /2 and *2 for each arc
# global edge = Array{Int64}(undef, 0, 2)
# global N = 50 #100
# global origin = 1
# global destination = N
# global Len = 0 
function print2File(cL, cU, stat, myFile)
    println(stat)
    println(myFile)
    outfile = "./N50_"*stat*"UncRange_"*myFile*".jl"
    println(outfile)
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(f,"edge = ", edge) #length(df[:CELL]))
    println(f,"cL_orig = ", cL)
    println(f,"cU_orig = ", cU)
    println(f,"d = ", d)
    println(f,"Len = length(cL_orig)")

    println(f,"c_orig = 0.5*(cL_orig+cU_orig)")

    #STARTING SOLUTION:
    println(f,"yy = zeros(Len)")
    println(f,"yy = ", yy)
    println(f,"SP_init = sum(yy[i]*c_orig[i] for i = 1:Len)")
    println(f,"p = [1.0]")
    println(f,"g = [SP_init]")
    println(f,"h = [0.0]")

    println(f,"origin = 1")
    println(f,"destination = 50")

    println(f,"last_node = maximum(edge)")
    println(f,"all_nodes = collect(1:last_node)")

    println(f,"M_orig = zeros(Len)")

    println(f,"for i = 1:Len")
    println(f,"    M_orig[i] = cU_orig[i] - cL_orig[i]")
    println(f,"end")


    println(f,"delta1 = ", delta1)
    println(f,"delta2 = ", delta2)
    println(f, "b = ", b) #rand(1:b_max))
    
#     println(f, )
    println(f,"last_node = maximum(edge)")
    println(f,"β = ", β)
    close(f)
end
function divideby2(cL, cU, M, myFile)
    c_bar = 0.5.*(cL + cU)
    M = 0.25.*M
    cL = c_bar - M
    cU = c_bar + M
    print2File(cL, cU, "Half", myFile)
#     println("\ncL = ", cL[M.>0])
#     println("cU = ", cU[M.>0])
end
function multby2(cL, cU, M, myFile)
    c_bar = 0.5.*(cL + cU)
#     M = 0.25.*M
    cL = c_bar - M
    cU = c_bar + M
    A_neg = findall(cL.<0)
#     println("\nA_neg = ", A_neg)
    for a in A_neg
        cU[a] = cU[a]-cL[a]
        cL[a] = 0
    end
    print2File(cL, cU, "Double", myFile)
#     println(cL[A_neg])
#     println("\ncL = ", cL[M.>0])
#     println("cU = ", cU[M.>0])
end

for ins in [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    include("./N50/N50_"*myFile*".jl")
#     println("cL = ", cL_orig)
#     println("M_orig = ", M_orig)
    uncA = findall(M_orig.>0)
    println("density = ", length(uncA)/length(edge[:,1]))
#     println("uncA (", length(uncA),") = ", uncA)
#     println("cL_orig = ", cL_orig[M_orig.>0])
#     println("cU_orig = ", cU_orig[M_orig.>0])
    
    divideby2(cL_orig, cU_orig, M_orig,myFile)
    multby2(cL_orig, cU_orig, M_orig, myFile)
#     println("M_orig = ", M_orig[M_orig.>0])
end

density = 0.19838056680161945
Half
16
./N50_HalfUncRange_16.jl
Double
16
./N50_DoubleUncRange_16.jl
density = 0.20175438596491227
Half
35
./N50_HalfUncRange_35.jl
Double
35
./N50_DoubleUncRange_35.jl
density = 0.1984126984126984
Half
41
./N50_HalfUncRange_41.jl
Double
41
./N50_DoubleUncRange_41.jl
density = 0.1981981981981982
Half
43
./N50_HalfUncRange_43.jl
Double
43
./N50_DoubleUncRange_43.jl
density = 0.20168067226890757
Half
56
./N50_HalfUncRange_56.jl
Double
56
./N50_DoubleUncRange_56.jl
density = 0.19838056680161945
Half
79
./N50_HalfUncRange_79.jl
Double
79
./N50_DoubleUncRange_79.jl
density = 0.20072992700729927
Half
84
./N50_HalfUncRange_84.jl
Double
84
./N50_DoubleUncRange_84.jl
density = 0.20175438596491227
Half
110
./N50_HalfUncRange_110.jl
Double
110
./N50_DoubleUncRange_110.jl
density = 0.1991701244813278
Half
118
./N50_HalfUncRange_118.jl
Double
118
./N50_DoubleUncRange_118.jl
density = 0.20175438596491227
Half
142
./N50_HalfUncRange_142.jl
Double
142
./N50_DoubleUncRang

In [58]:
#Change budget to b=4 and b=10 for each set:
function updateBudgetb(myFile,b_new)
    println("b_new = ", b_new)
    println(myFile)
    outfile = "./N50_b"*string(b_new)*"_"*myFile*".jl"
    println(outfile)
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(f,"edge = ", edge) #length(df[:CELL]))
    println(f,"cL_orig = ", cL_orig)
    println(f,"cU_orig = ", cU_orig)
    println(f,"d = ", d)
    println(f,"Len = length(cL_orig)")

    println(f,"c_orig = 0.5*(cL_orig+cU_orig)")

    #STARTING SOLUTION:
    println(f,"yy = zeros(Len)")
    println(f,"yy = ", yy)
    println(f,"SP_init = sum(yy[i]*c_orig[i] for i = 1:Len)")
    println(f,"p = [1.0]")
    println(f,"g = [SP_init]")
    println(f,"h = [0.0]")

    println(f,"origin = ", origin)
    println(f,"destination = ", destination)

    println(f,"last_node = maximum(edge)")
    println(f,"all_nodes = collect(1:last_node)")

    println(f,"M_orig = zeros(Len)")

    println(f,"for i = 1:Len")
    println(f,"    M_orig[i] = cU_orig[i] - cL_orig[i]")
    println(f,"end")


    println(f,"delta1 = ", delta1)
    println(f,"delta2 = ", delta2)
    println(f, "b = ", b_new) #rand(1:b_max))
    
#     println(f, )
    println(f,"last_node = maximum(edge)")
    println(f,"β = ", β)
    close(f)
end

for ins in [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    include("./N50/N50_"*myFile*".jl")
#     println("density = ", length(uncA)/length(edge[:,1]))
    b_new = 4
    updateBudgetb(myFile, b_new)
    b_new = 10
    updateBudgetb(myFile, b_new)
end

b_new = 4
16
./N50_b4_16.jl
b_new = 10
16
./N50_b10_16.jl
b_new = 4
35
./N50_b4_35.jl
b_new = 10
35
./N50_b10_35.jl
b_new = 4
41
./N50_b4_41.jl
b_new = 10
41
./N50_b10_41.jl
b_new = 4
43
./N50_b4_43.jl
b_new = 10
43
./N50_b10_43.jl
b_new = 4
56
./N50_b4_56.jl
b_new = 10
56
./N50_b10_56.jl
b_new = 4
79
./N50_b4_79.jl
b_new = 10
79
./N50_b10_79.jl
b_new = 4
84
./N50_b4_84.jl
b_new = 10
84
./N50_b10_84.jl
b_new = 4
110
./N50_b4_110.jl
b_new = 10
110
./N50_b10_110.jl
b_new = 4
118
./N50_b4_118.jl
b_new = 10
118
./N50_b10_118.jl
b_new = 4
142
./N50_b4_142.jl
b_new = 10
142
./N50_b10_142.jl


In [77]:

set = rand(1:200, 10)
for ins in set # [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    dataSet = "N100"
    include("./"*dataSet*"/"*dataSet*"_"*myFile*".jl")
#     println("cL = ", cL_orig)
#     println("M_orig = ", M_orig)
    uncA = findall(M_orig.>0)
    println(ins," .UncNum = ", length(uncA)/length(edge[:,1]))
    println("\t density = ", length(d)/destination/(destination-1))
    println("\t maxM = ", maximum(M_orig))
    println("\t min c = ", minimum(cL_orig))
    println("\t b = ", b)
end

89 .UncNum = 0.20041536863966772
	 density = 0.09727272727272727
	 maxM = 20.0
	 min c = 0.0
	 b = 7
170 .UncNum = 0.19978858350951373
	 density = 0.09555555555555556
	 maxM = 20.0
	 min c = 4.0
	 b = 7
150 .UncNum = 0.20039880358923232
	 density = 0.10131313131313131
	 maxM = 20.0
	 min c = 0.0
	 b = 7
139 .UncNum = 0.19959266802443992
	 density = 0.09919191919191919
	 maxM = 20.0
	 min c = 0.0
	 b = 7
89 .UncNum = 0.20041536863966772
	 density = 0.09727272727272727
	 maxM = 20.0
	 min c = 0.0
	 b = 7
1 .UncNum = 0.20020855057351408
	 density = 0.09686868686868687
	 maxM = 20.0
	 min c = 2.0
	 b = 7
66 .UncNum = 0.19979188345473464
	 density = 0.09707070707070707
	 maxM = 20.0
	 min c = 0.0
	 b = 7
28 .UncNum = 0.19957983193277312
	 density = 0.09616161616161616
	 maxM = 20.0
	 min c = 1.0
	 b = 7
16 .UncNum = 0.2
	 density = 0.09848484848484848
	 maxM = 20.0
	 min c = 0.0
	 b = 7
188 .UncNum = 0.20040281973816718
	 density = 0.1003030303030303
	 maxM = 20.0
	 min c = 0.0
	 b = 7
